### FOURSQUARE

In [35]:
import requests 
import pandas as pd 
import numpy as np 
import random 
import folium 
from geopy.geocoders import Nominatim # Convierte una dirección en latitude y longitud


### Guardando las credenciales en variables

In [36]:
CLIENT_ID = 'U1LHGA0A24XNVNYN4FQJK3MD1VCBZ0KPP2LO12XQTS3FCN21' # ID
CLIENT_SECRET = 'N5HHZVFVPRVU2A3BMHFXU2UW1ODJ5QIM4PUQM0UJSCFIYRNR' # Secret ID
VERSION = '20190101'
LIMIT = 30
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: U1LHGA0A24XNVNYN4FQJK3MD1VCBZ0KPP2LO12XQTS3FCN21
CLIENT_SECRET:N5HHZVFVPRVU2A3BMHFXU2UW1ODJ5QIM4PUQM0UJSCFIYRNR


#### Suponemos que estamos en el hotel Conrad (NYC) entonces convertimos la dirección del hotel en latitud y longitud

In [37]:
address = '102 North End Ave, New York, NY 10282, Estados Unidos'

geolocator = Nominatim(user_agent="foursquare_agent") #A través de foursquare transforma la dirección en latitud y longitud
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [38]:
location

Location(Loopy Doopy Rooftop Bar, 102, North End Avenue, Battery Park City, Manhattan, Manhattan Community Board 1, New York County, New York, 10282, United States of America, (40.7149555, -74.0153365, 0.0))

## Busqueda por categoria de venue:

`https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Supongamos que vamos a buscar comida italiana (Italian food) alrededor del hotel Conrad (500 metros alrededor)

In [39]:
search_query = 'Italian food'
radius = 1000

#### Creamos el URL

In [40]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=U1LHGA0A24XNVNYN4FQJK3MD1VCBZ0KPP2LO12XQTS3FCN21&client_secret=N5HHZVFVPRVU2A3BMHFXU2UW1ODJ5QIM4PUQM0UJSCFIYRNR&ll=40.7149555,-74.0153365&v=20190101&query=Italian food&radius=1000&limit=30'

#### Hacer el request

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2c0a30fa01662f62e39f2b'},
 'response': {'venues': [{'id': '49bc3b0af964a52020541fe3',
    'name': 'Whole Foods Market',
    'location': {'address': '270 Greenwich Street',
     'crossStreet': 'at Warren St',
     'lat': 40.715579155420606,
     'lng': -74.01136823958119,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.715579155420606,
       'lng': -74.01136823958119}],
     'distance': 341,
     'postalCode': '10007',
     'cc': 'US',
     'neighborhood': 'Tribeca',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['270 Greenwich Street (at Warren St)',
      'New York, NY 10007',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      '

#### Obtener la información importante del JSON y convertirlo a un dataframe

In [52]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
#print(venues)
#tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.shape

(30, 24)

In [53]:
dataframe.columns

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address',
       'location.crossStreet', 'location.lat', 'location.lng',
       'location.labeledLatLngs', 'location.distance', 'location.postalCode',
       'location.cc', 'location.neighborhood', 'location.city',
       'location.state', 'location.country', 'location.formattedAddress',
       'delivery.id', 'delivery.url', 'delivery.provider.name',
       'delivery.provider.icon.prefix', 'delivery.provider.icon.sizes',
       'delivery.provider.icon.name'],
      dtype='object')

In [56]:
dataframe.head(3)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,49bc3b0af964a52020541fe3,Whole Foods Market,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1596721684,False,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",...,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",NaN,NaN,NaN,NaN,NaN,NaN
1,4bbb7d5aafe1b713ac3f2f4b,Food Bank For New York City,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1596721684,False,39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",...,New York,NY,United States,"[39 Broadway Fl 10, New York, NY 10006, United...",NaN,NaN,NaN,NaN,NaN,NaN
2,4c7e9152d860b60c514c5e9d,Ali's Fast Cooking Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1596721684,False,River Street,at Newark Avenue,40.711753,-74.014672,"[{'label': 'display', 'lat': 40.71175296524837...",...,Hoboken,NJ,United States,"[River Street (at Newark Avenue), Hoboken, NJ ...",NaN,NaN,NaN,NaN,NaN,NaN


#### Limpiamos el dataframe por información que nos interesa

In [57]:
#solamente dejamos las columnas name, categories, id y todas las que comienzan con "location"
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
filtered_columns

['name',
 'categories',
 'location.address',
 'location.crossStreet',
 'location.lat',
 'location.lng',
 'location.labeledLatLngs',
 'location.distance',
 'location.postalCode',
 'location.cc',
 'location.neighborhood',
 'location.city',
 'location.state',
 'location.country',
 'location.formattedAddress',
 'id']

In [58]:
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered

,name,categories,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,id
0,Whole Foods Market,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",341,10007,US,Tribeca,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",49bc3b0af964a52020541fe3
1,Food Bank For New York City,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",980,10006,US,NaN,New York,NY,United States,"[39 Broadway Fl 10, New York, NY 10006, United...",4bbb7d5aafe1b713ac3f2f4b
2,Ali's Fast Cooking Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",River Street,at Newark Avenue,40.711753,-74.014672,"[{'label': 'display', 'lat': 40.71175296524837...",360,07030,US,NaN,Hoboken,NJ,United States,"[River Street (at Newark Avenue), Hoboken, NJ ...",4c7e9152d860b60c514c5e9d
3,Rafael's Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",Barclay Street,Washington Street,40.716152,-74.010856,"[{'label': 'display', 'lat': 40.71615153012879...",400,10007,US,NaN,New York,NY,United States,"[Barclay Street (Washington Street), New York,...",50350113e4b04416d6a575c8
4,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,NaN,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,NaN,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
5,Plaza Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",1 Centre St,NaN,40.713627,-74.003236,"[{'label': 'display', 'lat': 40.71362706685846...",1031,10007,US,NaN,New York,NY,United States,"[1 Centre St, New York, NY 10007, United States]",4dee21fc18385379de543dd6
6,Food Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",Greenwich Street,Barclay Street,40.714182,-74.012413,"[{'label': 'display', 'lat': 40.714182, 'lng':...",261,NaN,US,NaN,New York,NY,United States,"[Greenwich Street (Barclay Street), New York, ...",51378818e4b079558b0a6a61
7,Pisillo Italian Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,40.710493,-74.007546,"[{'label': 'display', 'lat': 40.71049280792626...",823,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",588a4316326c5a4b60559f17
8,Food & Wine Magazine,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",225 Liberty St,NaN,40.711875,-74.015287,"[{'label': 'display', 'lat': 40.711875, 'lng':...",342,NaN,US,NaN,New York,NY,United States,"[225 Liberty St, New York, NY, United States]",56904b7e498eaf25c7535429
9,South Food Truck Court,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",NaN,Liberty,40.711932,-74.015537,"[{'label': 'display', 'lat': 40.711932, 'lng':...",336,NaN,US,NaN,New York,NY,United States,"[Liberty, New York, NY, United States]",51e57352498eb9115b270d65


In [59]:
dataframe_filtered["categories"][0]

[{'id': '4bf58dd8d48988d118951735',
  'name': 'Grocery Store',
  'pluralName': 'Grocery Stores',
  'shortName': 'Grocery Store',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
   'suffix': '.png'},
  'primary': True}]

In [102]:
# funcion que extrae la categoria del venue
def get_category_type(row):
    try:
        categories_list = row['categories']    
    except:
        categories_list = row['venue.categories']    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [61]:
# filtramos la categoria de cada fila
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered

,name,categories,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,id
0,Whole Foods Market,Grocery Store,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",341,10007,US,Tribeca,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",49bc3b0af964a52020541fe3
1,Food Bank For New York City,Office,39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",980,10006,US,NaN,New York,NY,United States,"[39 Broadway Fl 10, New York, NY 10006, United...",4bbb7d5aafe1b713ac3f2f4b
2,Ali's Fast Cooking Food Truck,Food Truck,River Street,at Newark Avenue,40.711753,-74.014672,"[{'label': 'display', 'lat': 40.71175296524837...",360,07030,US,NaN,Hoboken,NJ,United States,"[River Street (at Newark Avenue), Hoboken, NJ ...",4c7e9152d860b60c514c5e9d
3,Rafael's Food Truck,Food Truck,Barclay Street,Washington Street,40.716152,-74.010856,"[{'label': 'display', 'lat': 40.71615153012879...",400,10007,US,NaN,New York,NY,United States,"[Barclay Street (Washington Street), New York,...",50350113e4b04416d6a575c8
4,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,NaN,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,NaN,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
5,Plaza Food Court,Food Court,1 Centre St,NaN,40.713627,-74.003236,"[{'label': 'display', 'lat': 40.71362706685846...",1031,10007,US,NaN,New York,NY,United States,"[1 Centre St, New York, NY 10007, United States]",4dee21fc18385379de543dd6
6,Food Cart,Food Truck,Greenwich Street,Barclay Street,40.714182,-74.012413,"[{'label': 'display', 'lat': 40.714182, 'lng':...",261,NaN,US,NaN,New York,NY,United States,"[Greenwich Street (Barclay Street), New York, ...",51378818e4b079558b0a6a61
7,Pisillo Italian Cafe,Café,NaN,NaN,40.710493,-74.007546,"[{'label': 'display', 'lat': 40.71049280792626...",823,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",588a4316326c5a4b60559f17
8,Food & Wine Magazine,Office,225 Liberty St,NaN,40.711875,-74.015287,"[{'label': 'display', 'lat': 40.711875, 'lng':...",342,NaN,US,NaN,New York,NY,United States,"[225 Liberty St, New York, NY, United States]",56904b7e498eaf25c7535429
9,South Food Truck Court,Food Truck,NaN,Liberty,40.711932,-74.015537,"[{'label': 'display', 'lat': 40.711932, 'lng':...",336,NaN,US,NaN,New York,NY,United States,"[Liberty, New York, NY, United States]",51e57352498eb9115b270d65


In [62]:
#Limpiamos los nombres de las columnas dejando solo el ultimo elemento
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Whole Foods Market,Grocery Store,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",341,10007,US,Tribeca,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",49bc3b0af964a52020541fe3
1,Food Bank For New York City,Office,39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",980,10006,US,NaN,New York,NY,United States,"[39 Broadway Fl 10, New York, NY 10006, United...",4bbb7d5aafe1b713ac3f2f4b
2,Ali's Fast Cooking Food Truck,Food Truck,River Street,at Newark Avenue,40.711753,-74.014672,"[{'label': 'display', 'lat': 40.71175296524837...",360,07030,US,NaN,Hoboken,NJ,United States,"[River Street (at Newark Avenue), Hoboken, NJ ...",4c7e9152d860b60c514c5e9d
3,Rafael's Food Truck,Food Truck,Barclay Street,Washington Street,40.716152,-74.010856,"[{'label': 'display', 'lat': 40.71615153012879...",400,10007,US,NaN,New York,NY,United States,"[Barclay Street (Washington Street), New York,...",50350113e4b04416d6a575c8
4,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,NaN,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,NaN,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
5,Plaza Food Court,Food Court,1 Centre St,NaN,40.713627,-74.003236,"[{'label': 'display', 'lat': 40.71362706685846...",1031,10007,US,NaN,New York,NY,United States,"[1 Centre St, New York, NY 10007, United States]",4dee21fc18385379de543dd6
6,Food Cart,Food Truck,Greenwich Street,Barclay Street,40.714182,-74.012413,"[{'label': 'display', 'lat': 40.714182, 'lng':...",261,NaN,US,NaN,New York,NY,United States,"[Greenwich Street (Barclay Street), New York, ...",51378818e4b079558b0a6a61
7,Pisillo Italian Cafe,Café,NaN,NaN,40.710493,-74.007546,"[{'label': 'display', 'lat': 40.71049280792626...",823,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",588a4316326c5a4b60559f17
8,Food & Wine Magazine,Office,225 Liberty St,NaN,40.711875,-74.015287,"[{'label': 'display', 'lat': 40.711875, 'lng':...",342,NaN,US,NaN,New York,NY,United States,"[225 Liberty St, New York, NY, United States]",56904b7e498eaf25c7535429
9,South Food Truck Court,Food Truck,NaN,Liberty,40.711932,-74.015537,"[{'label': 'display', 'lat': 40.711932, 'lng':...",336,NaN,US,NaN,New York,NY,United States,"[Liberty, New York, NY, United States]",51e57352498eb9115b270d65


#### Veamos los nombres de los restauranes de comida italiana

In [63]:
dataframe_filtered.name

0                  Whole Foods Market
1         Food Bank For New York City
2       Ali's Fast Cooking Food Truck
3                 Rafael's Food Truck
4           Harry's Italian Pizza Bar
5                    Plaza Food Court
6                           Food Cart
7                Pisillo Italian Cafe
8                Food & Wine Magazine
9              South Food Truck Court
10                   Zak's Halal Food
11           Food & Wine Test Kitchen
12               Rafiqui's Halal Food
13                Breakfast food cart
14           Biryani House Food Truck
15                          Food Cart
16                Shafiq's Halal Food
17        Malaysia Kitchen Food Truck
18             Chambers St Halal Food
19    Conca Cucina Italian Restaurant
20             Halal Fast Food (cart)
21                         Halal Food
22     Adel's Best #1 Halal Food Cart
23                     Food Truck 777
24              Downtown Food Service
25                   City Food Market
26         A

In [64]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# aEl circulo rojo es el hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# los circulos azules son los restaurantes
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Explorar dado un Venue
`https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Obtengamos información del restaurante italiano más cercano: "Harry's Italian Pizza Bar" 

In [67]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=U1LHGA0A24XNVNYN4FQJK3MD1VCBZ0KPP2LO12XQTS3FCN21&client_secret=N5HHZVFVPRVU2A3BMHFXU2UW1ODJ5QIM4PUQM0UJSCFIYRNR&v=20190101'

#### Hacemos el request

In [68]:
result = requests.get(url).json()
result['response']['venue']

{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'n

In [69]:
print(result['response']['venue'].keys())

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


In [70]:
print(result['response']['venue']["contact"]["phone"]) #Obtener el telefono

2126081007


### Obtener el rating

In [72]:
try:
    print(result['response']['venue']['rating'])
except:
    print('El restaurante no ha sido ranqueado aún')

6.9


Revisamos el rating del segundo restaurante más cercano

In [75]:
venue_id = '4f3232e219836c91c7bfde94' # ID de Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('El restaurante no ha sido ranqueado aún')

El restaurante no ha sido ranqueado aún


Revisamos el rating del tercer restaurante

In [76]:
venue_id = '3fd66200f964a520f4e41ee3' # ID Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


### Obtenemos los likes

In [82]:
result['response']['venue']["likes"]["summary"]

'29 Likes'

## Explorando un lugar
`https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Vamos a explorar que venues existen alrededor del restaurante Ecco

In [83]:
latitude = 40.715337
longitude = -74.008848

#### Definir URL

In [84]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=U1LHGA0A24XNVNYN4FQJK3MD1VCBZ0KPP2LO12XQTS3FCN21&client_secret=N5HHZVFVPRVU2A3BMHFXU2UW1ODJ5QIM4PUQM0UJSCFIYRNR&ll=40.715337,-74.008848&v=20190101&radius=1000&limit=30'

#### Hacemos el request

In [95]:
results = requests.get(url).json()
'Existen {} venues cerca de Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'Existen 30 venues cerca de Ecco restaurant.'

In [97]:
results["response"]["groups"][0]['items']

#### Obtenemos la parte importante del JSON

In [98]:
items = results['response']['groups'][0]['items']

In [99]:
dataframe.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.delivery.id', 'venue.delivery.url',
       'venue.delivery.provider.name', 'venue.delivery.provider.icon.prefix',
       'venue.delivery.provider.icon.sizes',
       'venue.delivery.provider.icon.name', 'venue.photos.count',
       'venue.photos.groups', 'venue.location.crossStreet',
       'venue.location.neighborhood', 'venue.venuePage.id'],
      dtype='object')

#### Process JSON and convert it to a clean dataframe

In [103]:
dataframe = pd.json_normalize(items) # pasamos el JSON a dataframe

# filtramos columnas
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filtramos la categoria para cada fila
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered

,venue.name,venue.categories,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.crossStreet,venue.location.neighborhood,venue.id
0,Los Tacos No. 1,Taco Place,136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng':...",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United Sta...",NaN,NaN,5d5f24ec09484500079aee00
1,Korin,Furniture / Home Store,57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839...",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007,...",Church St,Tribeca,4af5d65ff964a52091fd21e3
2,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.71365284530189...",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 100...",at Church St,NaN,4c154c9a77cea593c401d260
3,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051...",202,10007,US,New York,NY,United States,[83 Murray St (btwn Greenwich St & W Broadway)...,btwn Greenwich St & W Broadway,NaN,54148bc6498ea7bb8c05b70a
4,Takahachi,Sushi Restaurant,145 Duane St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374...",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), N...",btwn W Broadway & Church St,NaN,4a8f2f39f964a520471420e3
5,Whole Foods Market,Grocery Store,270 Greenwich Street,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",214,10007,US,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",at Warren St,Tribeca,49bc3b0af964a52020541fe3
6,Chambers Street Wines,Wine Shop,148 Chambers St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.71577306392837...",88,10007,US,New York,NY,United States,[148 Chambers St (btwn West Broadway & Hudson ...,btwn West Broadway & Hudson St,NaN,4adcf23cf964a520cc6221e3
7,Nish Nūsh,Falafel Restaurant,88 Reade St,40.715537,-74.007725,"[{'label': 'display', 'lat': 40.71553710116416...",97,10013,US,New York,NY,United States,"[88 Reade St (at Church St), New York, NY 1001...",at Church St,NaN,50ba9119e4b071a4bae6dc10
8,Weather Up,Cocktail Bar,159 Duane St,40.716741,-74.008666,"[{'label': 'display', 'lat': 40.71674084163369...",157,10013,US,New York,NY,United States,"[159 Duane St (btwn Hudson St. & W Broadway), ...",btwn Hudson St. & W Broadway,NaN,4cd89eeb6e8b5941660c64d2
9,Restaurant Marc Forgione,New American Restaurant,134 Reade St,40.716380,-74.009629,"[{'label': 'display', 'lat': 40.71637984317071...",133,10013,US,New York,NY,United States,"[134 Reade St (btwn Hudson and Greenwich St), ...",btwn Hudson and Greenwich St,NaN,48510cf9f964a520a5501fe3


In [104]:
# limpiamos los nombres de columnas
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Los Tacos No. 1,Taco Place,136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng':...",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United Sta...",NaN,NaN,5d5f24ec09484500079aee00
1,Korin,Furniture / Home Store,57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839...",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007,...",Church St,Tribeca,4af5d65ff964a52091fd21e3
2,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.71365284530189...",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 100...",at Church St,NaN,4c154c9a77cea593c401d260
3,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051...",202,10007,US,New York,NY,United States,[83 Murray St (btwn Greenwich St & W Broadway)...,btwn Greenwich St & W Broadway,NaN,54148bc6498ea7bb8c05b70a
4,Takahachi,Sushi Restaurant,145 Duane St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374...",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), N...",btwn W Broadway & Church St,NaN,4a8f2f39f964a520471420e3
5,Whole Foods Market,Grocery Store,270 Greenwich Street,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",214,10007,US,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",at Warren St,Tribeca,49bc3b0af964a52020541fe3
6,Chambers Street Wines,Wine Shop,148 Chambers St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.71577306392837...",88,10007,US,New York,NY,United States,[148 Chambers St (btwn West Broadway & Hudson ...,btwn West Broadway & Hudson St,NaN,4adcf23cf964a520cc6221e3
7,Nish Nūsh,Falafel Restaurant,88 Reade St,40.715537,-74.007725,"[{'label': 'display', 'lat': 40.71553710116416...",97,10013,US,New York,NY,United States,"[88 Reade St (at Church St), New York, NY 1001...",at Church St,NaN,50ba9119e4b071a4bae6dc10
8,Weather Up,Cocktail Bar,159 Duane St,40.716741,-74.008666,"[{'label': 'display', 'lat': 40.71674084163369...",157,10013,US,New York,NY,United States,"[159 Duane St (btwn Hudson St. & W Broadway), ...",btwn Hudson St. & W Broadway,NaN,4cd89eeb6e8b5941660c64d2
9,Restaurant Marc Forgione,New American Restaurant,134 Reade St,40.716380,-74.009629,"[{'label': 'display', 'lat': 40.71637984317071...",133,10013,US,New York,NY,United States,"[134 Reade St (btwn Hudson and Greenwich St), ...",btwn Hudson and Greenwich St,NaN,48510cf9f964a520a5501fe3


In [33]:
dataframe_filtered["categories"]

0                          Taco Place
1              Furniture / Home Store
2                              Bakery
3       Vegetarian / Vegan Restaurant
4                    Sushi Restaurant
5                       Grocery Store
6                           Wine Shop
7                  Falafel Restaurant
8                        Cocktail Bar
9             New American Restaurant
10                   Asian Restaurant
11                       Burger Joint
12                         Playground
13                American Restaurant
14                                Gym
15                American Restaurant
16                  French Restaurant
17                       Cocktail Bar
18               Gym / Fitness Center
19                        Coffee Shop
20                     Farmers Market
21                              Hotel
22                               Park
23    Molecular Gastronomy Restaurant
24                         Bagel Shop
25                       Dance Studio
26          

#### Let's visualize these items on the map around our location

In [105]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

# Restaurante - circulo rojo
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# venues - circulos azules 
for lat, lng, label in zip(dataframe_filtered.head(25).lat, dataframe_filtered.head(25).lng, dataframe_filtered.head(25).categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map